In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# open the first file
url = r"/home/youssef/Desktop/Jobs_Data/Data_Collecting/job_skills.csv"
df = pd.read_csv(url) 
df

,job_link,job_skills
0,https://www.linkedin.com/jobs/view/housekeeper...,"Building Custodial Services, Cleaning, Janitor..."
1,https://www.linkedin.com/jobs/view/assistant-g...,"Customer service, Restaurant management, Food ..."
2,https://www.linkedin.com/jobs/view/school-base...,"Applied Behavior Analysis (ABA), Data analysis..."
3,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Engineering, Project Controls, Sche..."
4,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Assembly, Point to point wiring, St..."
...,...,...
1296376,https://www.linkedin.com/jobs/view/community-a...,"Communication Skills, Time Management, Custome..."
1296377,https://www.linkedin.com/jobs/view/sr-it-analy...,"Windows SQL, EDI X12, Edifecs Platform, Health..."
1296378,https://www.linkedin.com/jobs/view/operations-...,"Adaptability, Communication, Digital Fluency, ..."
1296379,https://www.linkedin.com/jobs/view/float-patie...,"CNA, EMT, BLS, Medical Assistant, CPCT, LPN, R..."


In [3]:
df["job_skills"].value_counts()

job_skills
Front Counter, DriveThru, Outside Order Taker, Runner, Dining Room Attendant, Drinks, Make Up Table, Prep, Grill, Fry, Porter, Coaching, High school diploma/GED, Restaurant leadership experience, Food Handlers Certification                                                                                                                                                                                                                                                                                                                                                            169
Front Counter, DriveThru, Outside Order Taker, Runner, Dining Room Attendant, Drinks, Make Up Table, Prep, Grill, Fry, Porter, Coaching, High school diploma/GED, 1+ years of restaurant leadership experience, Food Handlers Certification                                                                                                                                                                                   

In [4]:
import re
chunk_size = 50000
cleaned_chunks = []

def process_skill(skill):
    if not isinstance(skill, str):
        return None

    skill = skill.lower().strip()
    skill = re.sub(r'[^a-z0-9 ]+', '', skill)
    skill = re.sub(r'\s+', ' ', skill)

    if len(skill) == 0 or len(skill) > 50:
        return None

    return skill

for chunk in pd.read_csv(url, chunksize=chunk_size):

    # Basic cleaning
    chunk['job_skills'] = (
        chunk['job_skills']
        .astype(str)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
    )

    # Split into list
    chunk['job_skills'] = chunk['job_skills'].str.split(',')

    # Clean safely
    def clean_skill_list(skills):
        if not isinstance(skills, list):
            return None

        cleaned = []
        for skill in skills:
            s = process_skill(skill)
            if s is not None:
                cleaned.append(s)

        return ', '.join(cleaned) if len(cleaned) > 0 else None

    chunk['job_skills'] = chunk['job_skills'].apply(clean_skill_list)

    cleaned_chunks.append(chunk)

# Combine all cleaned chunks into a single DataFrame
df = pd.concat(cleaned_chunks, ignore_index=True)

In [5]:
chunk_size = 50000
def process_skill(skill):
    if pd.isna(skill):
        return None

    skill = str(skill).lower().strip()

    skill = re.sub(r'[^a-z0-9 ]+', ' ', skill)
    skill = re.sub(r'\s+', ' ', skill)

    if len(skill) == 0:
        return None

    if len(skill) > 50:
        return None

    return skill


cleaned_chunks = []
for chunk in pd.read_csv(url, chunksize=chunk_size):

    chunk["job_skills"] = chunk["job_skills"].astype(str)

    chunk["job_skills"] = chunk["job_skills"].str.lower().str.strip()

    chunk["job_skills"] = chunk["job_skills"].str.split(",")

    chunk["job_skills"] = chunk["job_skills"].apply(
        lambda skills: [
            s for s in (process_skill(skill) for skill in skills)
            if s is not None
        ]
        if isinstance(skills, list)
        else None
    )

    chunk["job_skills"] = chunk["job_skills"].apply(
        lambda skills: ", ".join(sorted(set(skills)))
        if isinstance(skills, list) and len(skills) > 0
        else None
    )

    cleaned_chunks.append(chunk)

df = pd.concat(cleaned_chunks, ignore_index=True)

In [6]:
df["job_skills"].value_counts()

job_skills
coaching, dining room attendant, drinks, drivethru, food handlers certification, front counter, fry, grill, high school diploma ged, make up table, outside order taker, porter, prep, restaurant leadership experience, runner                                                                                                                                                                                                                                                                                217
1 years of restaurant leadership experience, coaching, dining room attendant, drinks, drivethru, food handlers certification, front counter, fry, grill, high school diploma ged, make up table, outside order taker, porter, prep, runner                                                                                                                                                                                                                                                                

In [11]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect("/home/youssef/Desktop/Jobs_Data/DWH_Database/Jobs_Data")

# Write DataFrame to database
df.to_sql(
    name="Job_Skills",        # table name
    con=conn,
    if_exists="replace",  # replace table if exists
    index=False
)

# Close connection
conn.close()